In [1]:
import os
import time
import json
from PIL import Image
import urllib.request
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.error import HTTPError

endpoint_url = "https://query.wikidata.org/sparql"
img_data = {}

# Requête pour récupérer les infos de l'entité donnée - Q144/Chien par défaut
def query(wd='Q144', limit=5):
    return f"""SELECT ?item ?itemLabel ?pic
                WHERE {{
                  ?item wdt:P31 wd:{wd}.
                  ?item wdt:P18 ?pic.
                  SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
                }}
                LIMIT {limit}"""

def get_results(endpoint_url, query):
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36'
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    
    # Boucle de tentative en cas d'erreur 429 Too Many Requests
    for i in range(3):
        try:
            return sparql.query().convert()
        except HTTPError as e:
            if e.response.status_code == 429:
                retry_after = int(e.response.headers.get('Retry-After', '90'))
                print(f"Too Many Requests. Tentative dans {retry_after} secondes.")
                sleep(retry_after)
            else:
                raise e
    
    raise Exception("Nombre maximum de tentatives pour atteindre Wikidata atteint.")

# Pour créer le dossier des images s'il n'existe pas
def createFolder(name='images'):
    if name not in os.listdir():
        os.mkdir(name)
    else:
        print('Dossier '+name+' déjà existant !')

# Pour vérifier si un dossier est vide - évitons de retélécharger des images inutilement.
def isEmpty(name='images'):
    if len(os.listdir(name)) == 0:
        return True
    return False

# Pour ajouter des éléments à la bdd
def addImg(wd, limit):
    results = get_results(endpoint_url, query(wd, limit))
    for result in results['results']['bindings']:
        name_img = result['item']['value'].split("/")[-1] + "-" + wd # nom donné aux images téléchargées "numéro d'image - numéro de catégorie"
        
        #img_data[name_img] = {'image': result['pic']['value']}
        
        img_data[name_img] = {}
        downloadImage(result['pic']['value'], name_img)
        

# Ajoute les images de toutes les entités spécifiées
def addAllImg(wd, limit):
    for w in wd:
        addImg(w, limit)

# Pour télécharger l'image d'un élément de la bdd
def downloadImage(url, name_img='img0'):
    full_path = 'images/' + name_img + '.jpg'
    urllib.request.urlretrieve(url, full_path)
    time.sleep(1)

# Si les images sont déjà téléchargées on initialise le tableau
def initAlreadyDownload(namefolder='images'):
    for image in os.listdir(namefolder):
        #img_data[image.split(".")[0]] = {'image': ""}
        img_data[image.split(".")[0]] = {}

# Pour télécharger toutes les images - limit par query
def downloadAllImages(wd=['Q144'], limit=5):
    createFolder()
    if not isEmpty():
        print('Images déjà téléchargées !')
        initAlreadyDownload()
        return False
    addAllImg(wd, limit)
    return True 

#Supprimer une image du dossier 
def removeImage(image):
    if os.path.isfile('images/'+image):
        os.remove('images/'+image)
        img_data.pop(image.split(".")[0])
    else:
        shutil.rmtree('images/'+image)

bdd = ['Q144','Q14660','Q23442','Q28803','Q3305213'] #Chien, drapeau, ile, sandwich, peinture,
downloadAllImages(wd=bdd,limit=25)
print(img_data)


{'Q20709260-Q144': {}, 'Q20818039-Q144': {}, 'Q21002371-Q144': {}, 'Q21978595-Q144': {}, 'Q23486479-Q144': {}, 'Q24876723-Q144': {}, 'Q25222380-Q144': {}, 'Q25351863-Q144': {}, 'Q27305072-Q144': {}, 'Q27920551-Q144': {}, 'Q28457690-Q144': {}, 'Q29578313-Q144': {}, 'Q30897794-Q144': {}, 'Q35995833-Q144': {}, 'Q37944309-Q144': {}, 'Q37944320-Q144': {}, 'Q54257654-Q144': {}, 'Q54962180-Q144': {}, 'Q55637324-Q144': {}, 'Q58138181-Q144': {}, 'Q58308611-Q144': {}, 'Q58827053-Q144': {}, 'Q58865521-Q144': {}, 'Q60764537-Q144': {}, 'Q61970530-Q144': {}, 'Q4485658-Q14660': {}, 'Q4485659-Q14660': {}, 'Q4485660-Q14660': {}, 'Q4485664-Q14660': {}, 'Q4485666-Q14660': {}, 'Q4485671-Q14660': {}, 'Q4485687-Q14660': {}, 'Q4485689-Q14660': {}, 'Q4485709-Q14660': {}, 'Q4485711-Q14660': {}, 'Q4485714-Q14660': {}, 'Q4485715-Q14660': {}, 'Q4485718-Q14660': {}, 'Q4485722-Q14660': {}, 'Q4485729-Q14660': {}, 'Q4485731-Q14660': {}, 'Q4485736-Q14660': {}, 'Q4485741-Q14660': {}, 'Q4485749-Q14660': {}, 'Q4485754-Q1

In [20]:
import os
import json
from PIL import Image
import urllib.request
from SPARQLWrapper import SPARQLWrapper, JSON
from urllib.error import HTTPError
import time
import shutil
import numpy as np
from sklearn.cluster import MiniBatchKMeans
import webcolors


#Obtenir les metas d'une image
def getMetaImg(name, imgfile):
    #clés Exif
    DATE_KEY = 36867
    MODEL_KEY = 272
    
    img_data[name].update({
        #'size': [getSizeImg(imgfile.size), imgfile.size],
        'size': getSizeImg(imgfile.size),
        'orientation': getOrientationImg(imgfile.size),
        'format': imgfile.format,
    })
    
    getColorsImg(name, imgfile)
    
    exif_data = imgfile._getexif()
    if exif_data is not None:
        if DATE_KEY in exif_data:
            img_data[name]['date'] = exif_data[DATE_KEY]
        if MODEL_KEY in exif_data:
            img_data[name]['model'] = exif_data[MODEL_KEY]

def getOrientationImg(size): #(largeur,hauteur)
    width,height = size
    return "landscape" if width > height else "portrait" if height > width else "squared"
    
def getSizeImg(size): #(largeur,hauteur)
    width,height = size
    return "large" if width*height>1920*1080 else "small" if width*height<640*480 else "medium"
    
def getColorsImg(name, imgfile, cluster_nbr=2, n_init=1):

    img_data[name]['colors'] = []

    # Redimensionner l'image pour accélérer le traitement
    imgfile = imgfile.resize((int(imgfile.size[0]/2), int(imgfile.size[1]/2)))

    # Convertir l'image en un tableau numpy
    img_array = np.array(imgfile)

    try:
        img_vector = img_array.reshape(-1, 3)

        clusters = MiniBatchKMeans(n_clusters=cluster_nbr, n_init=n_init, random_state=69).fit(img_vector) #Random state pour garder la meme seed ppour toutes les images

        for i, center in enumerate(clusters.cluster_centers_):
            rgb = tuple(map(int, center))
            color_name = RGBtoName(rgb)

            img_data[name]['colors'].append({'rgb': rgb, 'name': color_name})

    except Exception as e:
        print('Pas de couleur : {}'.format(e))
        img_data[name]['colors'] = [-1]

        
def RGBtoName(rgb):
    # chercher le nom de couleur correspondant au code RGB
    try:
        color_name = webcolors.rgb_to_name(rgb)
    except ValueError:
        # si le nom exact n'est pas trouvé, trouver le nom le plus proche
        min_colors = {}
        for key, name_ in webcolors.CSS3_HEX_TO_NAMES.items():
            r_c, g_c, b_c = webcolors.hex_to_rgb(key)
            rd = (r_c - rgb[0]) ** 2
            gd = (g_c - rgb[1]) ** 2
            bd = (b_c - rgb[2]) ** 2
            min_colors[(rd + gd + bd)] = name_
        color_name = min_colors[min(min_colors.keys())]
    return color_name

def openImgGetMeta(name):
    image_path = "images/" + name
    if not os.path.isfile(image_path):
        print(f"Impossible de trouver le fichier {image_path}")
        return False

    try:
        with open(image_path, "rb") as f:
            imgfile = Image.open(f)
            # Vérifie si le fichier est une image valide et si sa taille ne dépasse pas une certaine limite
            imgfile.verify()
            if imgfile.size[0] * imgfile.size[1] > 178956970:  # Limite de pixels de PIL
                print(f"L'image {name} est trop grande et a été ignorée.")
                return False

            name = name.split(".")[0]
            getMetaImg(name, imgfile)

        imgfile.close()
        return True
    except Exception as e:
        print(f"Erreur lors de l'ouverture de l'image {image_path}: {e}")
        removeImage(name)
        return False

#Obtenir les metas de toutes nos images    
def openGetMetaAllImg(namefolder='images'):
    for image in os.listdir(namefolder):
        if not image.startswith('.'):  # Ignore les fichiers et dossiers cachés
            openImgGetMeta(image)

openGetMetaAllImg()

Erreur lors de l'ouverture de l'image images/Q109353074-Q28803.jpg: 'NoneType' object has no attribute 'seek'


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'images/Q109353074-Q28803'

In [21]:
#Enregistre les metadonnées des images dans un JSON
def exportToJSON(data,name='img_data.json'):
    with open(name, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("done")
        
exportToJSON(data=img_data)

done


In [17]:
def openJSON(file="img_data.json"):
    # Ouvrir le fichier JSON en mode lecture (fermé automatiquement)
    with open(file, 'r') as f:
        data = json.load(f)
    return data